In [1]:
# general tools
import os
import sys
import time
import h5py
import random
from glob import glob

import numpy as np
from datetime import datetime, timedelta
from random import shuffle

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from keras_unet_collection import utils as k_utils

2023-01-05 21:55:30.172254: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

In [4]:
import re

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
def pos_mixer(TRAIN, L, a0=0, a1=0.2):
    data_shape = TRAIN.shape
    out = np.empty((L, data_shape[-1]))
    
    for i in range(L):
        inds = np.random.choice(data_shape[0], 2)
        a = np.random.uniform(a0, a1)
        out[i, :] = a*TRAIN[inds[0], :] + (1-a)*TRAIN[inds[1], :]
    return out

In [7]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

In [8]:
def verif_metric(VALID_target, Y_pred, ref):


    # fpr, tpr, thresholds = roc_curve(VALID_target.ravel(), Y_pred.ravel())
    # AUC = auc(fpr, tpr)
    # AUC_metric = 1 - AUC
    
    BS = np.mean((VALID_target.ravel() - Y_pred.ravel())**2)
    #ll = log_loss(VALID_target.ravel(), Y_pred.ravel())
    
    #print('{}'.format(BS))
    metric = BS

    return metric / ref

In [9]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

In [10]:
def feature_extract(filenames, lon_80km, lon_minmax, lat_80km, lat_minmax, elev_80km, elev_max):
    
    lon_out = []
    lat_out = []
    elev_out = []
    mon_out = []
    
    base_v3_s = datetime(2018, 7, 15)
    base_v3_e = datetime(2020, 12, 2)

    base_v4_s = datetime(2020, 12, 3)
    base_v4_e = datetime(2022, 7, 15)

    base_ref = datetime(2010, 1, 1)
    
    date_list_v3 = [base_v3_s + timedelta(days=day) for day in range(365+365+142)]
    date_list_v4 = [base_v4_s + timedelta(days=day) for day in range(365+180-151)]
    
    for i, name in enumerate(filenames):
        
        if 'v4' in name:
            date_list = date_list_v4
        else:
            date_list = date_list_v3
        
        nums = re.findall(r'\d+', name)
        indy = int(nums[-2])
        indx = int(nums[-3])
        day = int(nums[-4])
        day = date_list[day]
        month = day.month
        
        month_norm = (month - 1)/(12-1)
        
        lon = lon_80km[indx, indy]
        lat = lat_80km[indx, indy]

        lon = (lon - lon_minmax[0])/(lon_minmax[1] - lon_minmax[0])
        lat = (lat - lat_minmax[0])/(lat_minmax[1] - lat_minmax[0])

        elev = elev_80km[indx, indy]
        elev = elev / elev_max
        
        lon_out.append(lon)
        lat_out.append(lat)
        elev_out.append(elev)
        mon_out.append(month_norm)
        
    return np.array(lon_out), np.array(lat_out), np.array(elev_out), np.array(mon_out)

def create_model():

    
    IN_vec = keras.Input((128,))    
    X = IN_vec
    #
    X = keras.layers.Dense(1024, activity_regularizer=keras.regularizers.L2(1e-2))(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation("gelu")(X)
    #X = keras.layers.Activation("relu")(X)

    X = keras.layers.Dropout(0.4)(X)
    
    #
    X = keras.layers.Dense(512, activity_regularizer=keras.regularizers.L2(1e-2))(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation("gelu")(X)
    #X = keras.layers.Activation("relu")(X)
    
    X = keras.layers.Dropout(0.4)(X)
    
    X = keras.layers.Dense(128, activity_regularizer=keras.regularizers.L2(1e-2))(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation("gelu")(X)
    #X = keras.layers.Activation("relu")(X)
    
    OUT = X
    OUT = keras.layers.Dense(1, activation='sigmoid', bias_initializer=keras.initializers.Constant(-10))(OUT)

    model = keras.models.Model(inputs=IN_vec, outputs=OUT)
    
    return model

In [11]:
with h5py.File(save_dir+'HRRR_domain.hdf', 'r') as h5io:
    lon_3km = h5io['lon_3km'][...]
    lat_3km = h5io['lat_3km'][...]
    lon_80km = h5io['lon_80km'][...]
    lat_80km = h5io['lat_80km'][...]
    elev_3km = h5io['elev_3km'][...]
    land_mask_80km = h5io['land_mask_80km'][...]
    
grid_shape = land_mask_80km.shape

In [12]:
elev_80km = du.interp2d_wraper(lon_3km, lat_3km, elev_3km, lon_80km, lat_80km, method='linear')

elev_80km[np.isnan(elev_80km)] = 0
elev_80km[elev_80km<0] = 0
elev_max = np.max(elev_80km)

lon_80km_mask = lon_80km[land_mask_80km]
lat_80km_mask = lat_80km[land_mask_80km]

lon_minmax = [np.min(lon_80km_mask), np.max(lon_80km_mask)]
lat_minmax = [np.min(lat_80km_mask), np.max(lat_80km_mask)]

In [13]:
lead = 4

In [14]:
filepath_train = "/glade/scratch/ksha/DATA/NCAR_batch/"
filepath_valid = "/glade/scratch/ksha/DATA/NCAR_batch/"
filepath_test = "/glade/scratch/ksha/DATA/NCAR_batch_v4/"
filepath_vec = "/glade/work/ksha/NCAR/"

In [ ]:
filename_train_lead2 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/TRAIN*lead2.npy"))
filename_train_lead3 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/TRAIN*lead3.npy"))
filename_train_lead4 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/TRAIN*lead4.npy"))
filename_train_lead5 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/TRAIN*lead5.npy"))

In [ ]:
filename_train = filename_train_lead2 + filename_train_lead3 + filename_train_lead4 + filename_train_lead5 + filename_train_lead6

In [ ]:
IND_TRAIN_lead = np.load('/glade/work/ksha/NCAR/IND_TRAIN_lead.npy', allow_pickle=True)[()]
TRAIN_ind2 = IND_TRAIN_lead['lead2']
TRAIN_ind3 = IND_TRAIN_lead['lead3']
TRAIN_ind4 = IND_TRAIN_lead['lead4']
TRAIN_ind5 = IND_TRAIN_lead['lead5']

In [ ]:
data_lead2_p0 = np.load('{}TRAIN_pp15_pred_lead{}_part0_base.npy'.format(filepath_vec, 2), allow_pickle=True)[()]
data_lead2_p1 = np.load('{}TRAIN_pp15_pred_lead{}_part1_base.npy'.format(filepath_vec, 2), allow_pickle=True)[()]
data_lead2_p2 = np.load('{}TRAIN_pp15_pred_lead{}_part2_base.npy'.format(filepath_vec, 2), allow_pickle=True)[()]

data_lead3_p0 = np.load('{}TRAIN_pp15_pred_lead{}_part0_base.npy'.format(filepath_vec, 3), allow_pickle=True)[()]
data_lead3_p1 = np.load('{}TRAIN_pp15_pred_lead{}_part1_base.npy'.format(filepath_vec, 3), allow_pickle=True)[()]
data_lead3_p2 = np.load('{}TRAIN_pp15_pred_lead{}_part2_base.npy'.format(filepath_vec, 3), allow_pickle=True)[()]

data_lead4_p0 = np.load('{}TRAIN_pp15_pred_lead{}_part0_base.npy'.format(filepath_vec, 4), allow_pickle=True)[()]
data_lead4_p1 = np.load('{}TRAIN_pp15_pred_lead{}_part1_base.npy'.format(filepath_vec, 4), allow_pickle=True)[()]
data_lead4_p2 = np.load('{}TRAIN_pp15_pred_lead{}_part2_base.npy'.format(filepath_vec, 4), allow_pickle=True)[()]

data_lead5_p0 = np.load('{}TRAIN_pp15_pred_lead{}_part0_base.npy'.format(filepath_vec, 5), allow_pickle=True)[()]
data_lead5_p1 = np.load('{}TRAIN_pp15_pred_lead{}_part1_base.npy'.format(filepath_vec, 5), allow_pickle=True)[()]
data_lead5_p2 = np.load('{}TRAIN_pp15_pred_lead{}_part2_base.npy'.format(filepath_vec, 5), allow_pickle=True)[()]

In [ ]:
TRAIN_lead2 = np.concatenate((data_lead2_p0['y_vector'], data_lead2_p1['y_vector'], data_lead2_p2['y_vector']), axis=0)
TRAIN_lead3 = np.concatenate((data_lead3_p0['y_vector'], data_lead3_p1['y_vector'], data_lead3_p2['y_vector']), axis=0)
TRAIN_lead4 = np.concatenate((data_lead4_p0['y_vector'], data_lead4_p1['y_vector'], data_lead4_p2['y_vector']), axis=0)
TRAIN_lead5 = np.concatenate((data_lead5_p0['y_vector'], data_lead5_p1['y_vector'], data_lead5_p2['y_vector']), axis=0)

In [ ]:
TRAIN_lead2_y = np.concatenate((data_lead2_p0['y_true'], data_lead2_p1['y_true'], data_lead2_p2['y_true']), axis=0)
TRAIN_lead3_y = np.concatenate((data_lead3_p0['y_true'], data_lead3_p1['y_true'], data_lead3_p2['y_true']), axis=0)
TRAIN_lead4_y = np.concatenate((data_lead4_p0['y_true'], data_lead4_p1['y_true'], data_lead4_p2['y_true']), axis=0)
TRAIN_lead5_y = np.concatenate((data_lead5_p0['y_true'], data_lead5_p1['y_true'], data_lead5_p2['y_true']), axis=0)

In [ ]:
TRAIN_X = np.concatenate((TRAIN_lead2, TRAIN_lead3, TRAIN_lead4, TRAIN_lead5), axis=0)
TRAIN_Y = np.concatenate((TRAIN_lead2_y, TRAIN_lead3_y, TRAIN_lead4_y, TRAIN_lead5_y), axis=0)

In [27]:
TRAIN_256_pos = TRAIN_X[TRAIN_Y==1, :]
TRAIN_256_neg = TRAIN_X[TRAIN_Y==0, :]

**Valid**

In [28]:
filename_valid_lead2 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/VALID*lead2.npy"))
filename_valid_lead3 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/VALID*lead3.npy"))
filename_valid_lead4 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/VALID*lead4.npy"))
filename_valid_lead5 = sorted(glob("/glade/scratch/ksha/DATA/NCAR_batch_v3/VALID*lead5.npy"))

In [29]:
valid_lead2 = np.load('{}TEST_pp15_pred_lead2_base.npy'.format(filepath_vec), allow_pickle=True)[()]
valid_lead3 = np.load('{}TEST_pp15_pred_lead3_base.npy'.format(filepath_vec), allow_pickle=True)[()]
valid_lead4 = np.load('{}TEST_pp15_pred_lead4_base.npy'.format(filepath_vec), allow_pickle=True)[()]
valid_lead5 = np.load('{}TEST_pp15_pred_lead5_base.npy'.format(filepath_vec), allow_pickle=True)[()]

VALID_lead2 = valid_lead2['y_vector']
VALID_lead3 = valid_lead3['y_vector']
VALID_lead4 = valid_lead4['y_vector']
VALID_lead5 = valid_lead5['y_vector']

VALID_lead2_y = valid_lead2['y_true']
VALID_lead3_y = valid_lead3['y_true']
VALID_lead4_y = valid_lead4['y_true']
VALID_lead5_y = valid_lead5['y_true']

In [30]:
IND_VALID_lead = np.load('/glade/work/ksha/NCAR/IND_VALID_lead.npy', allow_pickle=True)[()]
VALID_ind2 = IND_VALID_lead['lead2']
VALID_ind3 = IND_VALID_lead['lead3']
VALID_ind4 = IND_VALID_lead['lead4']
VALID_ind5 = IND_VALID_lead['lead5']

In [31]:
VALID_X = np.concatenate((VALID_lead2, VALID_lead3, VALID_lead4, VALID_lead5), axis=0)
VALID_Y = np.concatenate((VALID_lead2_y, VALID_lead3_y, VALID_lead4_y, VALID_lead5_y), axis=0)

In [32]:
training_rounds = 30
seeds = [12342, 2536234, 98765, 473, 865, 7456, 69472, 3456357, 3425, 678,
         2452624, 5787, 235362, 67896, 98454, 12445, 46767, 78906, 345, 8695, 
         2463725, 4734, 23234, 884, 2341, 362, 5, 234, 483, 785356, 23425]

In [37]:
ref = np.sum(VALID_Y) / len(VALID_Y)

# =========== Model Section ========== #

batch_dir = '/glade/scratch/ksha/DATA/NCAR_batch/'
temp_dir = '/glade/work/ksha/NCAR/Keras_models/'

key = 'STN_Lead{}'.format(lead)

model_name = '{}'.format(key)
model_path = temp_dir+model_name

tol = 0

# ========== Training loop ========== #
L_pos = len(TRAIN_256_pos)
L_neg = len(TRAIN_256_neg)

record = 1.1
print("Initial record: {}".format(record))

min_del = 0
max_tol = 15 # early stopping with patience

epochs = 500
batch_size = 64
L_train = 16 #int(len(TRAIN_Y_pick) / batch_size)

for r in range(training_rounds):
    if r == 0:
        tol = 0
    else:
        tol = -200

    model = create_model()
    #
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=keras.optimizers.Adam(lr=1e-4))
    
    # W_old = k_utils.dummy_loader('/glade/work/ksha/NCAR/Keras_models/BASE_Lead2/')
    # model.set_weights(W_old)
    
    set_seeds(int(seeds[r]))
    print('Training round {}'.format(r))

    for i in range(epochs):
        # if i > 0:
        #     backend.set_value(model.optimizer.learning_rate, decayed_learning_rate(i))
            
        start_time = time.time()

        # loop of batch
        for j in range(L_train):
            N_pos = 32
            N_neg = batch_size - N_pos

            ind_neg = du.shuffle_ind(L_neg)
            ind_pos = du.shuffle_ind(L_pos)

            ind_neg_pick = ind_neg[:N_neg]
            ind_pos_pick = ind_pos[:N_pos]

            X_batch_neg = TRAIN_256_neg[ind_neg_pick, :]
            X_batch_pos = TRAIN_256_pos[ind_pos_pick, :]
            X_batch = np.concatenate((X_batch_neg, X_batch_pos), axis=0)

            Y_batch = np.ones([batch_size,])
            Y_batch[:N_neg] = 0.0

            ind_ = du.shuffle_ind(batch_size)

            X_batch = X_batch[ind_, :]
            Y_batch = Y_batch[ind_]

            # train on batch
            model.train_on_batch(X_batch, Y_batch);

        # epoch end operations
        Y_pred = model.predict([VALID_X])

        Y_pred[Y_pred<0] = 0
        Y_pred[Y_pred>1] = 1

        record_temp = verif_metric(VALID_Y, Y_pred, ref)

        # if i % 10 == 0:
        #     model.save(model_path_backup)

        if (record - record_temp > min_del):
            print('Validation loss improved from {} to {}'.format(record, record_temp))
            record = record_temp
            tol = 0
            
            #print('tol: {}'.format(tol))
            # save
            print('save to: {}'.format(model_path))
            model.save(model_path)
        else:
            print('Validation loss {} NOT improved'.format(record_temp))
            if record_temp > 1.0:
                print('Early stopping')
                break;
            else:
                tol += 1
                if tol >= max_tol:
                    print('Early stopping')
                    break;
                else:
                    continue;
        print("--- %s seconds ---" % (time.time() - start_time))

Initial record: 1.1
Training round 0
Validation loss improved from 1.1 to 0.9997043158606339
save to: /glade/work/ksha/NCAR/Keras_models/STN_Lead4
INFO:tensorflow:Assets written to: /glade/work/ksha/NCAR/Keras_models/STN_Lead4/assets
--- 19.876680612564087 seconds ---
Validation loss improved from 0.9997043158606339 to 0.9996820541642476
save to: /glade/work/ksha/NCAR/Keras_models/STN_Lead4
INFO:tensorflow:Assets written to: /glade/work/ksha/NCAR/Keras_models/STN_Lead4/assets
--- 18.770101308822632 seconds ---
Validation loss improved from 0.9996820541642476 to 0.9996557828189909
save to: /glade/work/ksha/NCAR/Keras_models/STN_Lead4
INFO:tensorflow:Assets written to: /glade/work/ksha/NCAR/Keras_models/STN_Lead4/assets
--- 19.23515748977661 seconds ---
Validation loss 0.9996605960305971 NOT improved
Validation loss improved from 0.9996557828189909 to 0.999621864958134
save to: /glade/work/ksha/NCAR/Keras_models/STN_Lead4
INFO:tensorflow:Assets written to: /glade/work/ksha/NCAR/Keras_mod

KeyboardInterrupt: 

In [51]:
model = create_model()

model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=keras.optimizers.Adam(lr=0))

W_old = k_utils.dummy_loader('/glade/work/ksha/NCAR/Keras_models/BASE_Lead2/')
model.set_weights(W_old)

In [52]:
#VALID_Lead2_tune3

In [53]:
# ref = np.sum(TEST_Y[:405032]) / len(TEST_Y[:405032])
# Y_pred = model.predict([TEST_merge, TEST_256])
# record_temp = verif_metric(TEST_Y[:405032], Y_pred[:405032], ref)

In [54]:
ref = np.sum(VALID_Y) / len(VALID_Y)
Y_pred = model.predict([VALID_X, VALID_merge])
record_temp = verif_metric(VALID_Y, Y_pred, ref)

In [55]:
record_temp

0.8514755852660806

In [56]:
save_dict = {}
save_dict['Y_pred'] = Y_pred
save_dict['VALID_Y'] = VALID_Y
np.save('{}RESULT_BASE_lead{}_base.npy'.format(filepath_vec, lead), save_dict)
print('{}RESULT_BASE_lead{}_base.npy'.format(filepath_vec, lead))

/glade/work/ksha/NCAR/RESULT_BASE_lead2_base.npy
